<a href="https://colab.research.google.com/github/kynthesis/HaystackResearch/blob/main/WebSearch_LFQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Cách xây dựng một hệ thống QA lấy thông tin từ web search**

# 1. Kiểm tra GPU runtime

In [ ]:
!nvidia-smi

Fri Aug 18 05:44:50 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 2. Cài đặt Haystack

In [ ]:
%%bash

pip install --upgrade pip
pip install farm-haystack[colab,inference]

# 3. Nhập API của Serper và OpenAI

In [ ]:
from getpass import getpass

serper_api_key = getpass("Enter Serper API key:")
openai_api_key = getpass("Enter OpenAI API key:")

Enter Serper API key:··········
Enter OpenAI API key:··········


# 4. Import các Node cần thiết

In [ ]:
from haystack.nodes import PromptNode, PromptTemplate, TopPSampler
from haystack.nodes.retriever.web import WebRetriever
from haystack.pipelines import WebQAPipeline

# 5. Setup Prompt và Khởi tạo Pipeline

In [ ]:
prompt_text = """
Synthesize a comprehensive answer from the following most relevant paragraphs and the given question.
Provide a clear and concise response that summarizes the key points and information presented in the paragraphs.
Your answer should be in your own words and be no longer than 50 words.
\n\n Paragraphs: {documents} \n\n Question: {query} \n\n Answer:
"""

prompt_node = PromptNode(
    "text-davinci-003", default_prompt_template=PromptTemplate(prompt_text), api_key=openai_api_key, max_length=256
)

web_retriever = WebRetriever(api_key=serper_api_key, top_search_results=5, mode="preprocessed_documents", top_k=30)

pipeline = WebQAPipeline(retriever=web_retriever, prompt_node=prompt_node, sampler=TopPSampler(top_p=0.8))

# 6. Chuẩn bị câu hỏi cho hệ thống QA

In [ ]:
questions = [
    "What are the differences between ChatGPT-3 and ChatGPT-4?",
    "When should Extractive QA and Generative QA be used?",
    "What are the differences between Langchain and Haystack?"
]

# 7. Lọc các thông báo lỗi từ thư viện (optional)

In [ ]:
import logging

logger = logging.getLogger("haystack.nodes.retriever.link_content")
logger.setLevel(logging.CRITICAL)
logger = logging.getLogger("boilerpy3")
logger.setLevel(logging.CRITICAL)

# 8. Tiến hành thực hiện tác vụ QA và Nhận câu trả lời

In [ ]:
for q in questions:
    print(f"Question: {q}")
    response = pipeline.run(query=q)
    print(f"Answer: {response['results'][0]}")

Question: What are the differences between ChatGPT-3 and ChatGPT-4?
Answer: ChatGPT-4 is more powerful than ChatGPT-3, with more parameters, better understanding of context, domain-wise expertise, multi-language capabilities, and the ability to process images as well as text. It has a greater capacity to process information, a higher accuracy of responses, and fewer unsafe responses.
Question: When should Extractive QA and Generative QA be used?
Answer: Extractive QA should be used when the answer is present in a given context, such as a text, table, or HTML. Generative QA can be used when the answer must be generated based on a context, or when no context is provided and the answer must be completely generated by a model.
Question: What are the differences between Langchain and Haystack?
Answer: LangChain and Haystack both offer different approaches to building language applications. LangChain uses LLMs and Agents to delegate actions to build powerful applications, while Haystack focu